# My project for Optimization Course

Multivariate linear regression implemented through solving an optimization problem using the gradient descent method and some of its variants.

The dataset used was taken from Kaggle.com and concerns house prices in the New York area. The idea is to perform a linear regression using the price as the dependent variable and the number of bedrooms, number of bathrooms, and the size of the house as independent variables.

In [ ]:

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("dataset/NY-House-Dataset.csv")
print(df.head())



Data Cleaning


In [ ]:
#ELIMINO LE COLONNE NON DESIDERATE

columns_to_drop = ["ADDRESS", "STATE", "MAIN_ADDRESS", "ADMINISTRATIVE_AREA_LEVEL_2", 
                   "LOCALITY", "STREET_NAME", "LONG_NAME", "FORMATTED_ADDRESS", 
                   "LATITUDE", "LONGITUDE","TYPE","SUBLOCALITY","BROKERTITLE"]
df_filtered = df.drop(columns=columns_to_drop)


#FILTRO GLI OUTLIER

vars = ["PRICE", "BEDS", "BATH", "PROPERTYSQFT"]
#df_filtered = df.copy()

# Filtra i valori anomali utilizzando il concetto di IQR
for var in vars:
    Q1 = df_filtered[var].quantile(0.25)
    Q3 = df_filtered[var].quantile(0.75)
    
    IQR = Q3 - Q1
    lower_limit = Q1 - 1.5 * IQR
    upper_limit = Q3 + 1.5 * IQR
    
    df_filtered = df_filtered[(df_filtered[var] >= lower_limit) & (df_filtered[var] <= upper_limit)]
    
print(df_filtered.head)
    


Regression Model:

$y = w_0 + w_1 x1 + w_2 x2 + w_3 x3$

In this case:

$PRICE = w_0 + w_1 BEDS + w_2 BATH + w_3 PROPERTYSQFT$



In [ ]:
def standardize(x):
    """Standardize the original data points (mean 0 and std dev 1)."""
    x = x - np.mean(x)
    x = x / np.std(x)
    return x

def build_model_data(x1, x2, x3, y):
    """Get regression data in matrix form with three predictors."""
    b = y  
    num_samples = len(b)  
    A = np.c_[np.ones(num_samples), x1, x2, x3]
    
    return A, b


PRICE=df_filtered.iloc[:,0]
BEDS =df_filtered.iloc[:,1]
BATH =df_filtered.iloc[:,2]
PROPERTYSQFT=df_filtered.iloc[:,3]

print (PRICE.head())
print(BEDS.head())
print(BATH.head())
print(PROPERTYSQFT.head())

A, b = build_model_data(standardize(BEDS), standardize(BATH),standardize(PROPERTYSQFT),standardize(PRICE))



1. Gradient descendent method

In [24]:
def calculate_objective(Axmb):
    """Calculate ||Ax - b||^2 for the vector Axmb = Ax - b"""
    # ***************************************************
    # YOUR CODE HERE
    obj = (Axmb**2).sum()
    # ***************************************************    
    return obj

def compute_gradient(A, x, b):
    """Compute the gradient and objective function."""
    # ***************************************************
    # YOUR CODE HERE
    Axmb = A.dot(x) - b
    grad = 2 * A.T.dot(Axmb)
    # ***************************************************
    return grad, Axmb

def gradient_descent(A, initial_x, b, max_iters, gamma):
    """Gradient descent algorithm."""
    # Define parameters to store x and objective func. values
    xs = [initial_x]
    objectives = []
    x = initial_x
    for n_iter in range(max_iters):
      
        # compute objective and gradient
        grad, Axmb = compute_gradient(A, x, b)
        obj = calculate_objective(Axmb)

        # ***************************************************
        # YOUR CODE HERE
        # update x by a gradient descent step
        x = x - gamma * grad
        # ***************************************************
        
        # store x and objective function value
        xs.append(x)
        objectives.append(obj)
        print("Gradient Descent({bi}/{ti}): objective={l:.5f}, x=[{w0:.5f},{w1:.5f}]".format(
              bi=n_iter, ti=max_iters - 1, l=obj, w0=x[0], w1=x[1]))

    return objectives, xs


test

In [25]:
# Define the parameters of the algorithm.
max_iters = 30
gamma = 0.001  # gamma = 0.1 does not converge

# Initialization
x_initial = np.zeros(A.shape[1])

# Start gradient descent.
gradient_objectives_naive, gradient_xs_naive = gradient_descent(A, x_initial, b, max_iters, gamma)

Gradient Descent(0/29): objective=3659.00000, x=[-0.00000,2.26195]
Gradient Descent(1/29): objective=136698.31760, x=[-0.00000,-35.16268]
Gradient Descent(2/29): objective=23003907.57993, x=[0.00000,467.27816]
Gradient Descent(3/29): objective=3966911264.15454, x=[-0.00000,-6141.81526]
Gradient Descent(4/29): objective=684263221158.83960, x=[0.00000,80643.94971]
Gradient Descent(5/29): objective=118031770228636.46875, x=[-0.00000,-1059011.77904]
Gradient Descent(6/29): objective=20359873817378720.00000, x=[0.00000,13908071.09638]
Gradient Descent(7/29): objective=3511973956195641344.00000, x=[-0.00000,-182661767.21582]
Gradient Descent(8/29): objective=605797526157898678272.00000, x=[0.00000,2399017309.56936]
Gradient Descent(9/29): objective=104496971686375373930496.00000, x=[-0.00001,-31507992046.72483]
Gradient Descent(10/29): objective=18025192611737807192326144.00000, x=[0.00009,413817250478.64618]
Gradient Descent(11/29): objective=3109253440076961720890818560.00000, x=[-0.00078,